# DMRG : Modern MPS (Matrix product state) version

Edited by [Min Long](https://quantummc.xyz/members/min-long/) in October 2023.

Revised by [ZYM](https://quantummc.xyz/members/ziyang-meng/) in February 2026.

This is a notebook on modern DMRG algorithm written in Matrix product states(MPS) formalism. The code shown in this notebook is adapted from Internet and revised by Min Long.

For further information, one can read the reference articles of this notebook by Schollwoeck [arXiv:1008.3477](https://arxiv.org/abs/1008.3477v2).

In [1]:
import numpy as np
import scipy
import scipy.sparse.linalg
import scipy.sparse as sparse
import math

## Work flow explaination

Similar as the block enlargement method, we are dealing with a Hamilonian with form
$$
H = H^L_{eff} \otimes H_i \otimes H^R_{eff}
$$
where $H^L_{eff}$ is the Hamiltonian of the left chain written in the basis of low energy eigenstate, this is achieved by contracting all the left tensors, as shown in the figure, like wise for $H^R_{eff}$.


![Alt text](https://github.com/DavidGoing/DMRG/blob/main/figures/workflow.png?raw=true)

After the diagonalization, we perform SVD to move to next site.


## Explaination of the variables
- W : Matrix product operator form of the Heisenberg chain (at all the $H_i$).
- A : Left canonical state.
- B : Right canonical state.
- E : Left vacum state to store the information of contraction from left at each step ($H^L_{eff}$ in each step, and update every time when SVD is performed).
- F : Right vacuum state with same use ($H^R_{eff}$ in each step, and update every time when SVD is performed).



In [ ]:

## initial E and F matrices for the left and right vacuum states
def initial_E(W):
    E = np.zeros((W.shape[0], 1, 1))
    E[0] = 1
    return E


def initial_F(W):
    F = np.zeros((W.shape[1], 1, 1))
    F[-1] = 1
    return F

### Contraction function

Basically, the contraction is performed step be step. At nth site, the contraction from left to right is performed as showned below:

![Alt text](https://github.com/DavidGoing/DMRG/blob/main/figures/contraction.png?raw=true)

The contraction from right to left is performed as showned below:

![Alt text](https://github.com/DavidGoing/DMRG/blob/main/figures/contraction_r_l.png?raw=true)

In [ ]:
def contract_from_left(W, A, E, B):
    # the einsum function doesn't appear to optimize the contractions properly,
    # so we split it into individual summations in the optimal order
    # return np.einsum("abst,sij,aik,tkl->bjl",W,A,E,B, optimize=True)
    Temp = np.einsum("sij,aik->sajk", A, E)
    Temp = np.einsum("sajk,abst->tbjk", Temp, W)
    return np.einsum("tbjk,tkl->bjl", Temp, B)

def contract_from_right(W, A, F, B):
    # the einsum function doesn't appear to optimize the contractions properly,
    # so we split it into individual summations in the optimal order
    # return np.einsum("abst,sij,bjl,tkl->aik",W,A,F,B, optimize=True)
    Temp = np.einsum("sij,bjl->sbil", A, F)
    Temp = np.einsum("sbil,abst->tail", Temp, W)
    return np.einsum("tail,tkl->aik", Temp, B)

## "Can you draw the from right by hand" ?

# construct the F-matrices for all sites except the first
def construct_F(Alist, MPO, Blist):
    F = [initial_F(MPO[-1])]

    for i in range(len(MPO) - 1, 0, -1):
        F.append(contract_from_right(MPO[i], Alist[i], F[-1], Blist[i]))
    return F


def construct_E(Alist, MPO, Blist):
    return [initial_E(MPO[0])]

def Expectation(AList, MPO, BList):
    E = [[[1]]]
    for i in range(0, len(MPO)):
        E = contract_from_left(MPO[i], AList[i], E, BList[i])
    return E[0][0][0]


#  A 5-site example

## From right to left (Recursive coding)
### 1. Setup and Initialization
Before starting the right-to-left sweep, we define our chain of 5 sites (indices 0, 1, 2, 3, 4).
The list `F` is initialized with the 'Right Vacuum' boundary.

* MPO Tensors: [W0, W1, W2, W3, W4]
* Initial State: F = [initial_F]

---

### 2. The `construct_F` Loop Trace
Loop: `for i in range(4, 0, -1):`
In each step, `F[-1]` is contracted with site `i` to grow the environment.


| Iteration | Site (i) | Operation (using F[-1]) | Physical Content | F List State |
| :--- | :--- | :--- | :--- | :--- |
| Start | — | — | Right Vacuum | [F_vac] |
| Step 1 | 4 | Contract(W4, F_vac) | Site 4 | [F_vac, F_4] |
| Step 2 | 3 | Contract(W3, F_4) | Sites 3 + 4 | [F_vac, F_4, F_34] |
| Step 3 | 2 | Contract(W2, F_34) | Sites 2 + 3 + 4 | [F_vac, F_4, F_34, F_234] |
| Step 4 | 1 | Contract(W1, F_234) | Sites 1 + 2 + 3 + 4 | [F_vac, F_4, F_34, F_234, F_1234] |

---

<br>

## From left to right
### 1. Setup and Initialization (Left-to-Right)
We define our chain of 5 sites (indices 0, 1, 2, 3, 4). The list `E` is initialized
with the 'Left Vacuum' boundary, which represents the vacuum state to the left of site 0.

* MPO Tensors: [W0, W1, W2, W3, W4]
* Initial State: E = [initial_E]

---

### 2. The `construct_E` Loop Trace
Loop: `for i in range(0, 4):`
In each step, `E[-1]` (the existing left wall) is contracted with site `i`
to grow the environment for the next site.


| Iteration | Site (i) | Operation (using E[-1]) | Physical Content | E List State |
| :--- | :--- | :--- | :--- | :--- |
| Start | — | — | Left Vacuum | [E_vac] |
| Step 1 | 0 | Contract(W0, E_vac) | Site 0 | [E_vac, E_0] |
| Step 2 | 1 | Contract(W1, E_0) | Sites 0 + 1 | [E_vac, E_0, E_01] |
| Step 3 | 2 | Contract(W2, E_01) | Sites 0 + 1 + 2 | [E_vac, E_0, E_01, E_012] |
| Step 4 | 3 | Contract(W3, E_012) | Sites 0 + 1 + 2 + 3 | [E_vac, E_0, E_01, E_012, E_0123] |

---


### Coarse graining and truncation function

To perform 2-site DMRG, we need to do coarse graining for states and operators.

For two site state, two site MPS or product of MPS, the idea of doing coarse graining is similar, we just view two bond at the same direction as one bond (with larger index), as illustrated in the figure below

![image.png](https://github.com/DavidGoing/DMRG/blob/main/figures/coarse_graining.png?raw=true)

In [ ]:

# 2-1 coarse-graining of two site MPO into one site
def coarse_grain_MPO(W, X):
    return np.reshape(np.einsum("abst,bcuv->acsutv", W, X),
                      [W.shape[0], X.shape[1],
                       W.shape[2] * X.shape[2],
                       W.shape[3] * X.shape[3]])

def product_W(W1, W2):
    return np.reshape(np.einsum("abst,cdtu->acbdsu", W1, W2), [W1.shape[0] * W2.shape[0],
                                                               W1.shape[1] * W2.shape[1],
                                                               W1.shape[2], W2.shape[3]])

def product_MPO(M1, M2):
    assert len(M1) == len(M2)
    Result = []
    for i in range(0, len(M1)):
        Result.append(product_W(M1[i], M2[i]))
    return Result

# 2-1 coarse-graining of two-site MPS into one site
def coarse_grain_MPS(A, B):
    return np.reshape(np.einsum("sij,tjk->stik", A, B),
                      [A.shape[0] * B.shape[0], A.shape[1], B.shape[2]])

#           (d1 * d2)
#               |
#    ---(χL)---[ A ]---(χR)---

#          (d1)     (d2)          <-- Physical legs split
#           |        |
#    ---(χL)---[ Θ ]---(χR)---    <-- Indices grouped as (0, 2) and (1, 3)

#      (d1)                (d2)
#       |                   |
#  ---(χL)[ U ]---(S)---(S)---[ V ](χR)---


def fine_grain_MPS(A, dims):
    assert A.shape[0] == dims[0] * dims[1]
    Theta = np.transpose(np.reshape(A, dims + [A.shape[1], A.shape[2]]),
                         (0, 2, 1, 3))
    M = np.reshape(Theta, (dims[0] * A.shape[1], dims[1] * A.shape[2]))
    U, S, V = np.linalg.svd(M, full_matrices=0)
    U = np.reshape(U, (dims[0], A.shape[1], -1))
    V = np.transpose(np.reshape(V, (-1, dims[1], A.shape[2])), (1, 0, 2))
    # assert U is left-orthogonal
    # assert V is right-orthogonal
    # print(np.dot(V[0],np.transpose(V[0])) + np.dot(V[1],np.transpose(V[1])))
    return U, S, V

def truncate_MPS(U, S, V, m):
    m = min(len(S), m)
    trunc = np.sum(S[m:])
    S = S[0:m]
    U = U[:, :, 0:m]
    V = V[:, 0:m, :]
    return U, S, V, trunc, m

### Work flow : Procedure of DMRG

Define the Hamiltonian based on the transformation of the vector instead of the matrix elements. It is a common way to define a large spase matrix. Then diagonalize the matrix using Lancoz method.

![image.png](https://github.com/DavidGoing/DMRG/blob/main/figures/workflow.png?raw=true)

From the diagram expression of the Hamiltonain, we see that this Hamiltonian transform tensors.

In [ ]:
class HamiltonianMultiply(sparse.linalg.LinearOperator):
    def __init__(self, E, W, F):
        self.E = E
        self.W = W
        self.F = F
        self.dtype = np.dtype('d')
        self.req_shape = [W.shape[2], E.shape[1], F.shape[1]]
        # self.req_shape = [W.shape[2], E.shape[1], F.shape[2]] ORIGINAL VERSION
        self.size = self.req_shape[0] * self.req_shape[1] * self.req_shape[2]
        self.shape = [self.size, self.size]

    def _matvec(self, A):
        # the einsum function doesn't appear to optimize the contractions properly,
        # so we split it into individual summations in the optimal order
        # R = np.einsum("aij,sik,abst,bkl->tjl",self.E,np.reshape(A, self.req_shape),
        #              self.W,self.F, optimize=True)
        R = np.einsum("aij,sik->ajsk", self.E, np.reshape(A, self.req_shape))
        R = np.einsum("ajsk,abst->bjtk", R, self.W)
        R = np.einsum("bjtk,bkl->tjl", R, self.F)
        return np.reshape(R, -1)


## optimize a single site given the MPO matrix W, and tensors E,F
def optimize_site(A, W, E, F):
    H = HamiltonianMultiply(E, W, F)
    # we choose tol=1E-8 here, which is OK for small calculations.
    # to bemore robust, we should take the tol -> 0 towards the end
    # of the calculation.
    E, V = sparse.linalg.eigsh(H, 1, v0=A, which='SA', tol=1E-8)
    return (E[0], np.reshape(V[:, 0], H.req_shape))


def optimize_two_sites(A, B, W1, W2, E, F, m, dir):
    W = coarse_grain_MPO(W1, W2)
    AA = coarse_grain_MPS(A, B)
    H = HamiltonianMultiply(E, W, F)
    E, V = sparse.linalg.eigsh(H, 1, v0=AA, which='SA')
    AA = np.reshape(V[:, 0], H.req_shape)
    A, S, B = fine_grain_MPS(AA, [A.shape[0], B.shape[0]])
    A, S, B, trunc, m = truncate_MPS(A, S, B, m)
    if (dir == 'right'):
        B = np.einsum("ij,sjk->sik", np.diag(S), B)
    else:
        assert dir == 'left'
        A = np.einsum("sij,jk->sik", A, np.diag(S))
    return E[0], A, B, trunc, m


def two_site_dmrg(MPS, MPO, m, sweeps):
    E = construct_E(MPS, MPO, MPS)
    F = construct_F(MPS, MPO, MPS)
    F.pop()
    for sweep in range(0, int(sweeps / 2)):
        for i in range(0, len(MPS) - 2):
            Energy, MPS[i], MPS[i + 1], trunc, states = optimize_two_sites(MPS[i], MPS[i + 1],
                                                                           MPO[i], MPO[i + 1],
                                                                           E[-1], F[-1], m, 'right')
            print("Sweep {:} Sites {:},{:}    Energy {:16.12f}    States {:4} Truncation {:16.12f}"
                  .format(sweep * 2, i, i + 1, Energy, states, trunc))
            E.append(contract_from_left(MPO[i], MPS[i], E[-1], MPS[i]))
            F.pop();
        for i in range(len(MPS) - 2, 0, -1):
            Energy, MPS[i], MPS[i + 1], trunc, states = optimize_two_sites(MPS[i], MPS[i + 1],
                                                                           MPO[i], MPO[i + 1],
                                                                           E[-1], F[-1], m, 'left')
            print("Sweep {} Sites {},{}    Energy {:16.12f}    States {:4} Truncation {:16.12f}"
                  .format(sweep * 2 + 1, i, i + 1, Energy, states, trunc))
            F.append(contract_from_right(MPO[i + 1], MPS[i + 1], F[-1], MPS[i + 1]))
            E.pop();
    return MPS


In [ ]:


d = 2  # local bond dimension
N = 100  # number of sites
D = 10 # virtue bond dimension

InitialA1 = np.zeros((d, 1, 1))
InitialA1[0, 0, 0] = 1
InitialA2 = np.zeros((d, 1, 1))
InitialA2[1, 0, 0] = 1

## initial state |01010101>
MPS = [InitialA1, InitialA2] * int(N / 2)

## Local operators
I = np.identity(2)
Z = np.zeros((2, 2))
Sz = np.array([[0.5, 0],
               [0, -0.5]])
Sp = np.array([[0, 0],
               [1, 0]])
Sm = np.array([[0, 1],
               [0, 0]])

## Hamiltonian MPO
W = np.array([[I, Sz, 0.5 * Sp, 0.5 * Sm, Z],
              [Z, Z, Z, Z, Sz],
              [Z, Z, Z, Z, Sm],
              [Z, Z, Z, Z, Sp],
              [Z, Z, Z, Z, I]])

Wfirst = np.array([[I, Sz, 0.5 * Sp, 0.5 * Sm, Z]])

Wlast = np.array([[Z], [Sz], [Sm], [Sp], [I]])

# the complete MPO
MPO = [Wfirst] + ([W] * (N - 2)) + [Wlast]

# 8 sweeps with m=10 states


### Test region

In [ ]:
print(np.shape(Wfirst))
print(np.shape(initial_F(Wlast)))
print(np.shape(initial_E(Wfirst)))

(1, 5, 2, 2)
(1, 1, 1)
(1, 1, 1)


### End test region

In [ ]:
HamSquared = product_MPO(MPO, MPO)

MPS = two_site_dmrg(MPS, MPO, D, 8)

Energy = Expectation(MPS, MPO, MPS)
print("Final energy expectation value {}".format(Energy))

H2 = Expectation(MPS, HamSquared, MPS)
print("variance = {:16.12f}".format(H2 - Energy * Energy))

Sweep 0 Sites 0,1    Energy -25.059016994375    States    2 Truncation   0.000000000000
Sweep 0 Sites 1,2    Energy -25.189692620786    States    2 Truncation   0.000000000000
Sweep 0 Sites 2,3    Energy -25.395800958670    States    2 Truncation   0.000000000000
Sweep 0 Sites 3,4    Energy -25.560586239348    States    2 Truncation   0.000000000000
Sweep 0 Sites 4,5    Energy -25.743669962857    States    2 Truncation   0.000000000000
Sweep 0 Sites 5,6    Energy -25.917602338044    States    2 Truncation   0.000000000000
Sweep 0 Sites 6,7    Energy -26.095754582728    States    2 Truncation   0.000000000000
Sweep 0 Sites 7,8    Energy -26.271867353000    States    2 Truncation   0.000000000000
Sweep 0 Sites 8,9    Energy -26.448938381096    States    2 Truncation   0.000000000000
Sweep 0 Sites 9,10    Energy -26.625551828355    States    2 Truncation   0.000000000000
Sweep 0 Sites 10,11    Energy -26.802381755193    States    2 Truncation   0.000000000000
Sweep 0 Sites 11,12    Energy